In [ ]:
from keras.models import Sequential
from keras import layers
from keras.utils import plot_model
from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras import optimizers
import tensorflow as tf

# Review: workflow

#### (1) 문제 정의와 데이터셋 수집 
- 무엇을 예측하려 하는가? 이를 예측하기 위한 training data가 있는가? 
- 예측하려는 문제의 종류는 무엇인가?
    - Binary classification
    - Multi-class classification 
    - Regression
- Examples
    - __숫자 손글씨의 인식 (0-9)__
    - 영화 리뷰의 감성 분류 (positive/negative)
    - 얼굴 사진의 감정 인식 (happy/unhappy)
    

#### (2) 성공 지표의 선택
- 최종 모형 비교를 위한 지표 선택 
- Binary classification: accuracy, ROC AUC, precision(=TP/(TP+FP)), recall(=TP/(TP+FN)), F1-measure 등
- Multi-class classification: average precision 등
- Regression: MSE, MAE
- Keras에서 학습 과정의 validation set에 대한 metric으로서 AUC, precision, recall 등을 제공하지 않음(왜? mini-batch에 대한 위의 지표는 오히려 방해가 될 수 있음): Test set에서 계산하는 것이 바람직



#### (3) 평가 방법 선택
- Hold-out validation set 사용
    - Train data의 일정 부분을 validation set으로 사용 
    - 데이터의 양이 많을 때 사용하는 방법
    - `keras.model.fit`의 `validation_data` 또는 `validation_split` option
- K-fold cross-validation
    - Train set을 K-개의 무작위 set으로 구분한 뒤 하나씩 validation set으로 사용하며 반복
    - Hold-out validation set을 구성하기에 데이터가 적을 때 사용  
    - Keras 자체의 cv 모듈이 없으므로 scikit-learn의 `KFold`를 사용 (참고: https://3months.tistory.com/321)


#### (4) 데이터 준비 
- Input data는 일반적으로 [-1,1] 혹은 [0,1] 사이의 데이터로 스케일 조정
- 사용하려는 모델에 맞는 input 형태로 조정
- 필요시 output data 형태 변환(ex. one-hot encoding)

In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

#### (5) Baseline보다 나은 모델 훈련 
- 이진분류 라면 정확도 0.5, MNIST 데이터라면 정확도 0.1보다 높은 모형 만들기 
- 적절한 layer 종류의 선택
    - Dense
    - Convolution
    - Dropout
    - Pooling
    - Recurrent
    - Embedding
- 마지막 layer의 activation function 선택
    - output의 형태에 따라 조정
    - Sigmoid, softmax, linear 등
- Loss function 선택
    - 풀고자 하는 문제의 종류에 따라 선택
    - binary_crossentropy, categorical_crossentropy, mse 등
    - 미분 가능해야 하고 주어진 mini-batch에서 계산 가능해야 함. (ROC AUC 등은 사용 불가) 
- Optimizer와 learning rate 선택 
    - rmsprop, adam과 default learning rate 사용이 무난 
    
<img src="figures/cheatsheet_loss.PNG" width="80%">

In [ ]:
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10, batch_size=512, validation_split=0.2)

In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
test_pred=model.predict(test_images)

#### (6) Scaling up: overfitting 모델 구축 
- 충분한 성능을 나타내는 모델을 만들기 위해 모형을 크게 확장 
    - Layer 추가 
    - 각 layer의 unit 추가 
    - Epoch 수 증가 
- train loss와 validation loss를 모니터 

#### (7) Regularization, hyperparameter tuning 
- 반복적으로 모델 수정, 훈련, 평가하며 모델 튜닝 
    - Dropout 추가 
    - Layer 추가 혹은 제거 
    - L1 또는 L2 penalty 추가 
    - Layer의 unit 수나 learning rate의 튜닝 
  

In [ ]:
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer=optimizers.adam(lr=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

import time 
now = time.strftime("%c")
callbacks_list = [
    TensorBoard(log_dir='./logs/mnist/'+now),
    ModelCheckpoint(filepath='./models/mnist.h5',monitor='val_loss',save_best_only=True)
]

model.fit(train_images, train_labels, epochs=100, batch_size=512, validation_split=0.2, callbacks=callbacks_list)

<font color=darkred>
    
#  Example 4-2. CNN for Hand Signs


손가락으로 숫자를 표현하는 사진들을 학습하여 나타내고 있는 숫자(0-6)를 예측하는 문제를 실습



<img src="figures/SIGNS.png" style="width:800px;height:300px;">



In [ ]:
!pip install h5py
#!pip install pillow

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.utils import plot_model, to_categorical
from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard
from keras import optimizers

import h5py
import numpy as np
import matplotlib.pyplot as plt
#from PIL import Image

### Loading data

In [ ]:
train_dataset = h5py.File('data/train_signs.h5', "r")

X_train_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
Y_train_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

test_dataset = h5py.File('data/test_signs.h5', "r")
X_test_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
Y_test_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

classes = np.array(test_dataset["list_classes"][:]) # the list of classes

In [ ]:
list(train_dataset.keys())

In [ ]:
train_dataset["train_set_x"][0].shape

In [ ]:
X_train_orig.shape, Y_train_orig.shape

In [ ]:
X_test_orig.shape, Y_test_orig.shape

In [ ]:
# Example of a picture
index = 0
plt.imshow(X_train_orig[index])
print ("y = " + str(Y_train_orig[index]))

### 데이터 준비

<font color=blue> 
TO DO: 데이터를 모델에 입력하기 적절한 형태로 변환하시오. model fitting 시 validation set과 training set으로 활용하기 위해 train data의 앞에서부터 200개를 `X_val, Y_val`으로 저장하고 나머지 train data를  `partial_X_train, partial_Y_train`으로 저장하시오.

In [ ]:
## Your answer comes here












### Baseline model 구축
<font color="blue">
TO DO: 상대적으로 간단한 layer, activation, loss, optimizer를 사용하여 모형을 구축하고 test accuracy를 통해 성능 확인하시오. Accuracy 1/6 이상의 적절한 성능이 나오는지 확인하시오.

In [ ]:
## Your answer comes here












### Hyperparameter tuning을 통한 모형 수정 

<font color="blue">

TO DO: 
- layer 수의 결정
- filter/node의 수와 learning rate을 인수로 하는 `build_model` 함수를 생성 
- `build_model` 함수를 사용하여 몇 가지 모형 구조에서 learning rate을 random search를 통해 tuning 시도
- test accuracy 기준으로 final model 선택 


In [ ]:
## Your answer comes here












### 나의 사진 테스트

<font color="blue">

TO DO: 
    본인의 손가락으로 숫자를 표현하는 사진을 찍어 jpg형태로 저장한 후 figures folder에 업로드하고 아래의 코드를 실행하여 사진을 모형에 입력가능한 형태로 변환하시오. 변환된 사진을 학습된 모형에 입력하여 분류 결과를 확인하시오. 

In [ ]:
from keras.preprocessing import image 

img_path = 'figures/myhand.jpg'
img = image.load_img(img_path, target_size=(64, 64))
plt.imshow(img)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.

In [ ]:
model.predict(x)

In [ ]:
model.predict_classes(x)

References
- https://www.coursera.org/specializations/deep-learning